In [1]:
import torch
from torch import nn
import numpy as np


In [2]:
fname_train = "training1.pt"
fname_class = "classes.pt"

In [3]:
training = torch.load(fname_train)
classes = torch.load(fname_class)
training_dataset = torch.utils.data.TensorDataset(training, classes)
train_loader = torch.utils.data.DataLoader(training_dataset, shuffle=True)
model = nn.Sequential(
    nn.Linear(20, 10),
    nn.Sigmoid(),
    nn.Linear(10, 7),
    nn.Softmax(dim=1)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in range(100):
    for i, (input, label) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()


<ipython-input-3-4c4c8a500ddf>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  training = torch.load(fname_train)
<ipython-input-3-4c4c8a500ddf>:2: FutureWarning: You are u

## 2.

In [4]:
fname_test = "test.pt"
test = torch.load(fname_test)
model.eval()
with torch.no_grad():
    outputs = model(test)

<ipython-input-4-e6eb8599173b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test = torch.load(fname_test)


# 3.

In [5]:
def germanplural(output_activations):
    # most informative of the row
    _, predicted_classes = torch.max(output_activations, dim=1)
    # numpy it
    predicted_classes = predicted_classes.numpy()
    # bare confusion matrix
    confusion_matrix = np.zeros((8, 7), dtype=int)
    # recreate test data organization to pass to confusion matrix
    model_classes = []
    # 10 from each class
    for i in range(7):
        model_classes.extend([i] * 10)
    # additional 100 for the last test set
    model_classes.extend([7] * 100)
    model_classes = np.array(model_classes)
    # for each class in model class
    # put them to a row according to the test data structure
    # put them to a column according to the prediction.
    for i in range(len(model_classes)):
        cur_class = model_classes[i]
        predicted_class = predicted_classes[i]
        if cur_class < 7:
            confusion_matrix[cur_class, predicted_class] += 1
        else:
            confusion_matrix[7, predicted_class] += 1
    return confusion_matrix

In [6]:
germanplural(outputs)
# Here's every row is trying to replicate test data structure.
# row 1 is class 1 prototypes, row 2 is class 2 prototypes.
# columns show how many in that column predicted to be class 1/2/3, etc.
# The column number reflects the predicted class number.

array([[ 9,  0,  0,  1,  0,  0,  0],
       [ 0, 10,  0,  0,  0,  0,  0],
       [ 0,  0, 10,  0,  0,  0,  0],
       [ 0,  0,  0, 10,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  0,  0],
       [ 0,  1,  0,  0,  0,  7,  2],
       [ 0,  0,  0,  0,  0,  0, 10],
       [ 9, 11, 14, 12, 22, 12, 20]])

# 4.

In [7]:
fname_train = "training2.pt"
fname_class = "classes.pt"
fname_test = "test.pt"
training = torch.load(fname_train)
classes = torch.load(fname_class)
training_dataset = torch.utils.data.TensorDataset(training, classes)
train_loader = torch.utils.data.DataLoader(training_dataset, shuffle=True)
model = nn.Sequential(
    nn.Linear(20, 10),
    nn.Sigmoid(),
    nn.Linear(10, 7),
    nn.Softmax(dim=1)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in range(100):
    for i, (input, label) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

test = torch.load(fname_test)
model.eval()
with torch.no_grad():
    outputs = model(test)

<ipython-input-7-ae524a1be451>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  training = torch.load(fname_train)
<ipython-input-7-ae524a1be451>:5: FutureWarning: You are u

In [8]:
print(germanplural(outputs))

[[ 4  0  6  0  0  0  0]
 [ 0 10  0  0  0  0  0]
 [ 0  0 10  0  0  0  0]
 [ 0  0  0 10  0  0  0]
 [ 0  0  0  0 10  0  0]
 [ 3  1  0  1  0  0  5]
 [ 0  0  0  0  0  0 10]
 [55  4  9  7 14  0 11]]


# 5.
Under which conditions would you expect a general purpose statistical learning mechanism to be able to acquire a minority default?

<br/>

**Answer**:
Given the results, I would expect general models like the one we trained to acquire a minority default in cases when the input has the inherent characteristics of randomness. The training data or its manipulated version should be such that there are no generalizations to be drawn from the minority default case.

<br/>

*Personal Thoughts*: This does not make a lot of sense to me. There is no way that German speakers or any other minority default-type language speaker just hear random sounds.

# 6.
Given that German has a minority default, what prediction would you make about the structure of the German lexicon?

<br/>

**Answer**: My hypothesis would be that the german lexicon and word formation is not based on analogy/comparison. The storage probably is not optimized for analogy or prototypicality relations. However, I feel like analogy still has a place in it, given the examples you have provided in the problem set pdf. It would be fair to say, even though analogy is inescapable, rule based connections is a must.


# 7.

Propose one other way in which the input data to the network could be manipulated, and make a prediction about how your proposed manipulation will affect learning performance in the network.

<br/>

**Answer**: Given the German language and the demographic of Germany, I feel like kids in Germany hear a lot of different words and sounds from minority languages like Turkish, Polish, etc. If one assumes that a kid parser deterministically think about or generates possible word forms as they hear or entertain their plural forms in addition to the ones in German, their training data's first row should look like something like the combination of training1 and training2.

In addition to this, there should be probably some noise in that training data as well, as well as their mapping.

Thirdly, the contexts those -s on Turkish words and German words, as well as other German endings, are going to be different. So, it would be smart to add some sort of feature weighing as well.

Lastly, piggybacking on the feature weighing, we can implement a learned bias towards -s. Maybe, after seeing bunch of random stuff, the evidence needed for other classes is pretty low, and the when a word cannot be placed in those classes, a german kid automatically choses -s class. In this modelling exercise, I am putting weights that I just come up with, but in a better model, one can do that using other values from a speech corpus, maybe like phonemic surprisal?

# 8.

In [9]:
# combine trainign1 and trainign2 first class fifty fifty
# I tried a lot with adding noise and randomly given contextual weights. But I was not able to do it properly.
# I think I almost did it with flags, but I failed.
# Thus, I am giving something that works, rather than that.
# I am sorry.

# But I was able to implement the feature weighing for the different classes.

fname_train1 = "training1.pt"
fname_train2 = "training2.pt"
fname_class = "classes.pt"

training1 = torch.load(fname_train1)
classes = torch.load(fname_class)
training2 = torch.load(fname_train2)

dataset1 = torch.utils.data.TensorDataset(training1, classes)
dataset2 = torch.utils.data.TensorDataset(training2, classes)

# separate class1s
class1_indices_dataset1 = (classes[:, 0] == 1).nonzero().squeeze()
class1_indices_dataset2 = (classes[:, 0] == 1).nonzero().squeeze()
class1_dataset1 = torch.utils.data.TensorDataset(training1[class1_indices_dataset1], classes[class1_indices_dataset1])
class1_dataset2 = torch.utils.data.TensorDataset(training2[class1_indices_dataset2], classes[class1_indices_dataset2])
half_class1_size = len(class1_dataset1) // 2

# get half of the class1s from different datasets
class1_half_dataset1 = torch.utils.data.TensorDataset(class1_dataset1.tensors[0][:half_class1_size], class1_dataset1.tensors[1][:half_class1_size])
class1_half_dataset2 = torch.utils.data.TensorDataset(class1_dataset2.tensors[0][:half_class1_size], class1_dataset2.tensors[1][:half_class1_size])

# put them together
combined_class1 = torch.utils.data.TensorDataset(
    torch.cat((class1_half_dataset1.tensors[0], class1_half_dataset2.tensors[0])),
    torch.cat((class1_half_dataset1.tensors[1], class1_half_dataset2.tensors[1]))
)

# get the non-class1 parts
non_class1_indices = (classes[:, 0] == 0).nonzero().squeeze()
non_class1_dataset = torch.utils.data.TensorDataset(training1[non_class1_indices], classes[non_class1_indices])
final_training_data = torch.cat((combined_class1.tensors[0], non_class1_dataset.tensors[0]))
final_classes = torch.cat((combined_class1.tensors[1], non_class1_dataset.tensors[1]))

# final dataset
final_dataset = torch.utils.data.TensorDataset(final_training_data, final_classes)



<ipython-input-9-3fe6d0258a3d>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  training1 = torch.load(fname_train1)
<ipython-input-9-3fe6d0258a3d>:14: FutureWarning: You a

In [10]:
fname_test = "test.pt"

training_dataset = final_dataset
train_loader = torch.utils.data.DataLoader(training_dataset, shuffle=True)
model = nn.Sequential(
    nn.Linear(20, 10),
    nn.Sigmoid(),
    nn.Linear(10, 7),
    nn.Softmax(dim=1)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in range(100):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)

        weights = torch.ones(7)
        weights[0] = 5.5
        weights[1:] *= 0.8
        weighted_criterion = nn.CrossEntropyLoss(weight=weights)

        loss = weighted_criterion(outputs, labels)
        loss.backward()
        optimizer.step()

test = torch.load(fname_test)
model.eval()
with torch.no_grad():
    outputs = model(test)

<ipython-input-10-75b9c51b70da>:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test = torch.load(fname_test)


In [11]:
print(germanplural(outputs))

[[10  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0]
 [ 0  0 10  0  0  0  0]
 [ 1  0  0  9  0  0  0]
 [ 0  0  0  0 10  0  0]
 [ 3  3  0  2  2  0  0]
 [10  0  0  0  0  0  0]
 [70  6  9  9  6  0  0]]


**Answer to 8**: Even though I was not able to implement the ability to track random-class-1s and prototypical class-1s as a way to implement contextual feature weighing, even the simple feature weighing of classes made the model slightly better.

The only problem now is that I am not able to predict the last 2 classes correctly. The antepenultimate class was always bad in all models. Yet the penultimate class is messed up simply due to my changes. I assume that slightly more data would fix it? I tried more data, which was just concetanating two datasets, instead of 50/50. Now, the results are better.

In [12]:

fname_train1 = "training1.pt"
fname_train2 = "training2.pt"
fname_class = "classes.pt"
fname_test = "test.pt"

training1 = torch.load(fname_train1)
classes = torch.load(fname_class)
training2 = torch.load(fname_train2)

dataset1 = torch.utils.data.TensorDataset(training1, classes)
dataset2 = torch.utils.data.TensorDataset(training2, classes)

combined_training_data = torch.cat((training1, training2), dim=0)
combined_classes = torch.cat((classes, classes), dim=0)
combined_dataset = torch.utils.data.TensorDataset(combined_training_data, combined_classes)

training_dataset = combined_dataset
train_loader = torch.utils.data.DataLoader(training_dataset, shuffle=True)

model = nn.Sequential(
    nn.Linear(20, 10),
    nn.Sigmoid(),
    nn.Linear(10, 7),
    nn.Softmax(dim=1)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in range(100):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)

        weights = torch.ones(7)
        weights[0] = 3.0
        weights[1:] *= 1.0

        weighted_criterion = nn.CrossEntropyLoss(weight=weights)

        loss = weighted_criterion(outputs, labels)
        loss.backward()
        optimizer.step()

test = torch.load(fname_test)
model.eval()
with torch.no_grad():
    outputs = model(test)

<ipython-input-12-6133ee59f11c>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  training1 = torch.load(fname_train1)
<ipython-input-12-6133ee59f11c>:7: FutureWarning: You a

In [13]:
germanplural(outputs)

array([[10,  0,  0,  0,  0,  0,  0],
       [ 0, 10,  0,  0,  0,  0,  0],
       [ 0,  0, 10,  0,  0,  0,  0],
       [ 0,  0,  0, 10,  0,  0,  0],
       [ 0,  0,  0,  0, 10,  0,  0],
       [ 3,  3,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0, 10],
       [64,  7,  7,  6,  9,  0,  7]])